# AutoGen - Bill Management Asistant
#### References
#### https://microsoft.github.io/autogen/0.2/docs/tutorial/introduction
#### https://microsoft.github.io/autogen/0.2/docs/tutorial/conversation-patterns#sequential-chats
#### https://microsoft.github.io/autogen/0.2/docs/tutorial/tool-use
#### https://microsoft.github.io/autogen/0.2/docs/reference/agentchat/contrib/multimodal_conversable_agent#multimodalconversableagent

In [10]:
import os

from autogen import (
    ConversableAgent,
    GroupChat,
    GroupChatManager,
)

from autogen.agentchat.contrib.multimodal_conversable_agent import (
    MultimodalConversableAgent
)

from IPython.display import Markdown, display

In [12]:
openai_api_key = "<API_KEY>"
os.environ["OPENAI_API_KEY"] = openai_api_key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

llm_config = {
    "config_list": [
        {
            "model": "gpt-4o", 
            "api_key": os.environ["OPENAI_API_KEY"],
        }
    ],
    "temperature": 0.2,
}

In [14]:
user_proxy = ConversableAgent(
    name="UserProxyAgent",
    system_message=(
        "You represent the user. You provide a public URL to a bill image "
        "and request expense extraction and analysis."
    ),
    llm_config=llm_config,
    human_input_mode="NEVER",
)

user_proxy.description = "Initiates bill processing by providing a public image URL."

In [16]:
bill_processing_agent = MultimodalConversableAgent(
    name="BillProcessingAgent",
    system_message=(
        "You are a bill processing agent.\n\n"
        "Given a bill image, extract:\n"
        "1. Merchant name\n"
        "2. Bill total\n"
        "3. Category (choose ONE strictly from: groceries, utilities, shopping, travel, other)\n"
        "4. Line items in the format: product | quantity | price\n\n"
        "Return structured, clear text suitable for downstream analysis."
    ),
    llm_config=llm_config,
    human_input_mode="NEVER",
)

bill_processing_agent.description = (
    "Extracts structured expense data and line items from bill images."
)

In [18]:
expense_summary_agent = ConversableAgent(
    name="ExpenseSummarizationAgent",
    system_message=(
        "You analyze extracted bill data and generate insights.\n\n"
        "Tasks:\n"
        "- Summarize total expenditure\n"
        "- Confirm category assignment\n"
        "- Highlight unusual or high spending\n"
        "- Produce a clean Markdown summary"
    ),
    llm_config=llm_config,
    human_input_mode="NEVER",
)

expense_summary_agent.description = (
    "Summarizes expenses and produces a final Markdown report."
)

In [20]:
group_chat = GroupChat(
    agents=[
        user_proxy,
        bill_processing_agent,
        expense_summary_agent,
    ],
    messages=[],
    max_round=5,
    send_introductions=True,
)

In [22]:
group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config,
)

In [34]:
def display_expense_summary(chat_result):
    best_content = None

    for msg in chat_result.chat_history:
        if msg.get("name") == "ExpenseSummarizationAgent":
            content = msg.get("content")
            if isinstance(content, str) and content.strip():
                if best_content is None or len(content) > len(best_content):
                    best_content = content.strip()

    if best_content:
        display(Markdown(best_content))
    else:
        print("No Expense Summary found.")

## Test case 1

In [28]:
image_url = "https://livbygrace.com/wp-content/uploads/Aldi-receipt-1.jpg"
chat_result = user_proxy.initiate_chat(
    group_chat_manager,
    message=(
        "Please process the following bill image and analyze expenses:\n\n"
        f"<img {image_url}>"
    ),
    summary_method="reflection_with_llm",
)

UserProxyAgent (to chat_manager):

Please process the following bill image and analyze expenses:

<img https://livbygrace.com/wp-content/uploads/Aldi-receipt-1.jpg>

--------------------------------------------------------------------------------

Next speaker: BillProcessingAgent

BillProcessingAgent (to chat_manager):

**Merchant Name:** ALDI  
**Bill Total:** $58.34  
**Category:** Groceries  

**Line Items:**

- 2% Milk | 1 | 2.60
- Large Eggs | 1 | 0.78
- Large Eggs | 1 | 0.78
- Mntry/Pepr Jack | 1 | 1.99
- SeasonedFrnchFries | 1 | 1.75
- Potato Puffs | 1 | 1.65
- Family Chkn Nugget | 1 | 4.99
- Carrots | 1 | 1.49
- Bananas LRW | 2.67 lb | 1.17
- Butter Quarters | 1 | 2.89
- Semi-Sweet Morsels | 1 | 1.89
- Vegetable Oil 48oz | 1 | 1.85
- Navel Oranges | 1 | 3.49
- Jumbo Biscuits | 1 | 1.29
- Kosher Baby Dills | 1 | 1.69
- Deli Mustards | 1 | 1.49
- Saltines | 1 | 0.75
- 72 ct. Baby Wipes | 1 | 1.59
- Fabric Soft. Sheet | 1 | 1.89
- Aluminum Foil | 1 | 2.29
- Taco Mix | 1 | 0.39
- 

In [36]:
display_expense_summary(chat_result)

### Expense Summary

**Total Expenditure:** $58.34  
**Category:** Groceries

### Line Items Breakdown

- **Dairy & Eggs:**
  - 2% Milk: $2.60
  - Large Eggs (2 units): $1.56
  - Butter Quarters: $2.89

- **Fruits & Vegetables:**
  - Carrots: $1.49
  - Bananas (2.67 lb): $1.17
  - Navel Oranges: $3.49
  - Cantaloupe: $0.99

- **Frozen Foods:**
  - Seasoned French Fries: $1.75
  - Potato Puffs: $1.65
  - Family Chicken Nugget: $4.99

- **Baking & Cooking:**
  - Semi-Sweet Morsels: $1.89
  - Vegetable Oil (48oz): $1.85
  - Tomato Paste: $0.29
  - Tomato Sauce: $0.29

- **Snacks & Condiments:**
  - Kosher Baby Dills: $1.69
  - Deli Mustards: $1.49
  - Saltines: $0.75
  - Taco Mix: $0.39

- **Bread & Bakery:**
  - Hamburger Buns: $0.99
  - Sandwich Wheat Bread: $0.85
  - Jumbo Biscuits: $1.29

- **Canned Goods:**
  - Cut Green Beans (2 units): $0.98
  - Sweet Peas (3 units): $1.47

- **Personal Care:**
  - Beauty Bar Soap: $1.99
  - Panty Liners: $3.69
  - Old Spice Deodorant: $2.92

- **Household Supplies:**
  - 72 ct. Baby Wipes: $1.59
  - Fabric Softener Sheets: $1.89
  - Aluminum Foil: $2.29

### Insights

- **High Spending Items:** 
  - Family Chicken Nugget: $4.99
  - Panty Liners: $3.69
  - Navel Oranges: $3.49

- **Unusual or Notable Purchases:**
  - Multiple purchases of Sweet Peas and Green Beans suggest stocking up on canned goods.
  - The variety of personal care items indicates a restock of essentials.

Overall, the spending aligns with typical grocery shopping, with a focus on stocking up on essentials and a few higher-cost items.

In [39]:
image_url2 = "https://liveandletsfly.com/wp-content/uploads/2018/05/Air-Europa-Fuel-Surcahrges-04-1024x489.png"
image_url3 = "https://static.cms.xcover.com/media/original_images/Final-Rental-Invoice-annotated_Y5HkQdn.png"
chat_result2 = user_proxy.initiate_chat(
    group_chat_manager,
    message=(
        "Please process the following bill image and analyze expenses:\n\n"
        f"<img {image_url2}>"
        f"<img {image_url3}>"
    ),
    summary_method="reflection_with_llm",
)
display_expense_summary(chat_result2)

UserProxyAgent (to chat_manager):

Please process the following bill image and analyze expenses:

<img https://liveandletsfly.com/wp-content/uploads/2018/05/Air-Europa-Fuel-Surcahrges-04-1024x489.png><img https://static.cms.xcover.com/media/original_images/Final-Rental-Invoice-annotated_Y5HkQdn.png>

--------------------------------------------------------------------------------

Next speaker: BillProcessingAgent

BillProcessingAgent (to chat_manager):

Here is the extracted information from the bill image:

**Merchant Name:** Thrifty Car Rental

**Bill Total:** $5657.00

**Category:** Travel

**Line Items:**

- Accident Damages Excess | 1 | $5000.00
- Claim Admin Fee | 1 | $55.00
- Credit Card Schrg | 1 | $68.48
- Location Fee @ 35% | 1 | $19.25
- GST @ 10% | 1 | $514.27

Please let me know if you need further assistance!

--------------------------------------------------------------------------------

Next speaker: ExpenseSummarizationAgent

ExpenseSummarizationAgent (to chat_manag

### Expense Summary

**Total Expenditure:** $5,657.00

**Category:** Travel

**Line Items:**
- **Accident Damages Excess:** $5,000.00
- **Claim Admin Fee:** $55.00
- **Credit Card Surcharge:** $68.48
- **Location Fee (35%):** $19.25
- **GST (10%):** $514.27

### Insights

- **High Spending Alert:** The "Accident Damages Excess" charge of $5,000.00 is notably high and constitutes the majority of the total bill. This should be reviewed to ensure it is accurate and justified.
- **Category Confirmation:** The expenses are correctly categorized under "Travel," as they pertain to car rental services.

This summary provides a clear breakdown of the expenses incurred and highlights significant charges for further review.